In [1]:
library(bigrquery)
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
# Store the project ID
projectid = "yhcr-prd-phm-bia-core"

In [3]:
# Set your query
sql <- "SELECT * FROM `yhcr-prd-phm-bia-core.CB_FDM_PrimaryCare_v5.tbl_SRCode` 
WHERE src_dateevent BETWEEN '2007-01-01' AND '2022-12-31' AND (src_ctv3code LIKE '%C1011%' OR
src_ctv3code LIKE '%C1031%' OR
src_ctv3code LIKE '%C1090%' OR
src_ctv3code LIKE '%C1091%' OR
src_ctv3code LIKE '%C1092%' OR
src_ctv3code LIKE '%C1093%' OR
src_ctv3code LIKE '%C1094%' OR
src_ctv3code LIKE '%C1095%' OR
src_ctv3code LIKE '%C1096%' OR
src_ctv3code LIKE '%C1097%' OR
src_ctv3code LIKE '%L1806%' OR
src_ctv3code LIKE '%X40J5%' OR
src_ctv3code LIKE '%X40J6%' OR
src_ctv3code LIKE '%XaELQ%' OR
src_ctv3code LIKE '%XaEnp%' OR
src_ctv3code LIKE '%XaEnq%' OR
src_ctv3code LIKE '%XaF05%' OR
src_ctv3code LIKE '%XaFWI%' OR
src_ctv3code LIKE '%XaFmA%' OR
src_ctv3code LIKE '%XaFn7%' OR
src_ctv3code LIKE '%XaFn8%' OR
src_ctv3code LIKE '%XaFn9%' OR
src_ctv3code LIKE '%XaIfG%' OR
src_ctv3code LIKE '%XaIfI%' OR
src_ctv3code LIKE '%XaIrf%' OR
src_ctv3code LIKE '%XaIzQ%' OR
src_ctv3code LIKE '%XaIzR%' OR
src_ctv3code LIKE '%XaJQp%' OR
src_ctv3code LIKE '%XaKyX%')"

# Run the query; this returns a bq_table object that you can query further
tb <- bq_project_query(projectid, sql)

# Store data in a tibble
t2d <-bq_table_download(tb)
# Print table
head(t2d)

Warning message in as.integer.integer64(x):
“NAs produced by integer overflow”


person_id,src_rowidentifier,src_dateeventrecorded,src_dateevent,src_iddoneby,src_idorganisationdoneat,src_ctv3code,src_ctv3text,src_snomedcode,src_numericvalue,⋯,src_idappointment,src_idvisit,care_site_id_doneat,care_site_id,care_site_id_registeredat,provider_id,procedure_source_concept_id,observation_source_concept_id,condition_source_concept_id,measurement_source_concept_id
<int>,<int>,<dttm>,<dttm>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
12460479,NA,2014-01-09 15:40:44,2014-01-09 15:40:44,207227,733375168560,X40J5,Type II diabetes mellitus,44054006,0,⋯,-1,-1,226,226,NA,NA,NA,NA,201826,NA
12479383,NA,2017-08-07 08:37:21,2012-10-15 00:00:00,-1,NULL,X40J5,Type II diabetes mellitus,44054006,0,⋯,-1,-1,NA,501,501,NA,NA,NA,201826,NA
12750799,NA,2009-02-27 10:45:32,2009-02-27 10:45:32,1524,B83019,X40J5,Type II diabetes mellitus,44054006,0,⋯,-1,-1,409,409,409,NA,NA,NA,201826,NA
12516315,285550315,2009-04-27 00:00:00,2008-12-08 00:00:04,604593,B83022,X40J5,Type II diabetes mellitus,44054006,0,⋯,-1,-1,82,82,82,NA,NA,NA,201826,NA
13679957,1233240668,2010-06-02 12:54:29,2010-05-13 00:00:00,533947,B83032,X40J5,Type II diabetes mellitus,44054006,0,⋯,-1,-1,841,841,841,NA,NA,NA,201826,NA
12801727,745086332,2009-07-19 00:00:00,2008-11-20 00:00:00,517869,B83067,X40J5,Type II diabetes mellitus,44054006,0,⋯,-1,-1,858,858,858,NA,NA,NA,201826,NA


In [4]:
# keep on columns 1,4,7 and 8
t2d <- t2d %>% select(1,4,7,8)
nrow(t2d)
sum(is.na(t2d))
sum(duplicated(t2d))

# remove duplicated rows and count remaining number of rows
t2d <- t2d[!duplicated(t2d),]
nrow(t2d)

# add indicator variable for t2d and remove ctv3 cols
t2d <- t2d %>%
  mutate(t2d_status = 1) %>%
  select(-3,-4)

# change class
t2d$src_dateevent <- as.Date(t2d$src_dateevent)
t2d$t2d_status <- as.integer(t2d$t2d_status)

[1] 175083

[1] 0

[1] 8198

[1] 166885

In [5]:
head(t2d)
sum(duplicated(t2d$person_id))

person_id,src_dateevent,t2d_status
<int>,<date>,<int>
12460479,2014-01-09,1
12479383,2012-10-15,1
12750799,2009-02-27,1
12516315,2008-12-08,1
13679957,2010-05-13,1
12801727,2008-11-20,1


[1] 110283

In [6]:
# choose earliest date of event per person
t2d <- t2d %>%
group_by(person_id) %>%
arrange(src_dateevent) %>%
mutate(event_number = row_number()) %>%
slice_min(event_number) %>%
select(-event_number)
nrow(t2d)

[1] 56602

In [7]:
# set destination table - do this regardless of whether it already exists
dest_dataset <- bq_dataset("yhcr-prd-phm-bia-core","CB_MYSPACE_AH")
dest_table <- bq_table(dest_dataset, "Type2Diabetes")

# create a full table spec as follows
# then pass this to bq_table_upload
dest_fields <- bq_fields(list(bq_field("person_id", "INT64", "REQUIRED"),
                              bq_field("src_dateevent", "DATE"),
                              bq_field("t2d_status", "INT64")
))

# delete an existing table if necessary
if(bq_table_exists(dest_table)) bq_table_delete(dest_table)
# upload file_data to dest_table
bq_table_upload(dest_table, t2d, fields = dest_fields, create_disposition='CREATE_IF_NEEDED', write_disposition='WRITE_TRUNCATE')

In [8]:
nrow(t2d)

[1] 56602